In [378]:
#import pytest
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from bs4 import BeautifulSoup
import pickle 

import os

from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
#from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from time import sleep

In [379]:
def get_team_quantities(driver):
    name = driver.find_element(By.XPATH, "/html/body/div[8]/div[3]/div[4]/div/div/div[1]/div[2]/h1/span").text[:-10]
    if len(driver.find_elements(By.XPATH, "/html/body/div[8]/div[3]/div[4]/div/div/div[1]/div[2]/div[1]/img")) != 0:
        country = driver.find_element(By.XPATH, "/html/body/div[8]/div[3]/div[4]/div/div/div[1]/div[2]/div[2]/div[1]/div/div[2]").text
        city_text = driver.find_element(By.XPATH, "/html/body/div[8]/div[3]/div[4]/div/div/div[1]/div[2]/div[2]/div[2]").text[2:]
        city = city_text[0:city_text.find("\n")]
    else:
        country = driver.find_element(By.XPATH, "/html/body/div[8]/div[3]/div[4]/div/div/div[1]/div[2]/div[1]/div[1]/div/div[2]").text
        city_text = driver.find_element(By.XPATH, "/html/body/div[8]/div[3]/div[4]/div/div/div[1]/div[2]/div[1]/div[2]").text[2:]
        city = city_text[0:city_text.find("\n")]
    display_name = driver.find_element(By.XPATH, "/html/body/div[8]/div[3]/div[5]/div/div/div[2]/div[2]/div/div[1]").text[5:]
    acronym = [char for char in display_name if char.isupper() ]
    acronym = ''.join(acronym)

    logo = driver.find_element(By.XPATH, "/html/body/div[8]/div[3]/div[4]/div/div/div[1]/div[1]/div/div/div/a/img").get_attribute("src")
    
    return [name, display_name, acronym, logo, country, city]

In [380]:
league_url = "https://www.zerozero.pt/edicao/liga-portugal-betclic-2024-25/187713"


In [382]:
service = Service(executable_path="/usr/local/bin/chromedriver-linux64/chromedriver")
driver = webdriver.Chrome(service=service)
driver.get(league_url)
sleep(1)

no_notifs = "/html/body/div[12]/div/div[3]/div/button[1]"
cookies = "/html/body/div[1]/div/div/div/div/div/div[2]/button[2]"

while len(driver.find_elements(By.XPATH, no_notifs)) == 0:
    sleep(0.1)
if len(driver.find_elements(By.XPATH, no_notifs)) != 0:
    button = driver.find_element(By.XPATH, no_notifs)
    button.click()
    sleep(0.5)
if len(driver.find_elements(By.XPATH, cookies)) != 0:
    button = driver.find_element(By.XPATH, cookies)
    button.click()
    sleep(0.5)


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=139.0.7258.138)
Stacktrace:
#0 0x5f273be161da <unknown>
#1 0x5f273b8bc780 <unknown>
#2 0x5f273b891790 <unknown>
#3 0x5f273b93c078 <unknown>
#4 0x5f273b95959c <unknown>
#5 0x5f273b933b73 <unknown>
#6 0x5f273b9006e3 <unknown>
#7 0x5f273b901331 <unknown>
#8 0x5f273bddc02b <unknown>
#9 0x5f273bddfe1e <unknown>
#10 0x5f273bdc2f99 <unknown>
#11 0x5f273bde09e8 <unknown>
#12 0x5f273bda792f <unknown>
#13 0x5f273be03788 <unknown>
#14 0x5f273be03966 <unknown>
#15 0x5f273be154f6 <unknown>
#16 0x70b82d09caa4 <unknown>
#17 0x70b82d129c3c <unknown>


In [ ]:
table_xpath = "/html/body/div[8]/div[3]/div[6]/div/div/div[1]/div[2]/table/tbody/tr/td/div[1]/div/table/tbody"
body = driver.find_element(By.XPATH, table_xpath)

rows = body.find_elements(By.TAG_NAME, "tr")
team_urls = [ rows[i].find_elements(By.TAG_NAME, "a")[1].get_attribute("href") for i in range(len(rows)) ]

driver.close()

In [ ]:
quants = []
#team_url = team_urls[5]
for team_url in team_urls:

    service = Service(executable_path="/usr/local/bin/chromedriver-linux64/chromedriver")
    driver = webdriver.Chrome(service=service)
    driver.get(team_url)
    sleep(1)

    no_notifs = "/html/body/div[12]/div/div[3]/div/button[1]"
    cookies = "/html/body/div[1]/div/div/div/div/div/div[2]/button[2]"
    while len(driver.find_elements(By.XPATH, no_notifs)) == 0:
        sleep(0.1)
    if len(driver.find_elements(By.XPATH, no_notifs)) != 0:
        button = driver.find_element(By.XPATH, no_notifs)
        button.click()
        sleep(0.5)
    if len(driver.find_elements(By.XPATH, cookies)) != 0:
        button = driver.find_element(By.XPATH, cookies)
        button.click()
        sleep(0.5)

    quants.append(get_team_quantities(driver))

    driver.close()

quants = np.array(quants)

In [ ]:
dict = {
    "id": np.arange(0, len(quants))+1,
    "display_name": quants[:, 0],
    "name": quants[:, 1],
    "acronym": quants[:,2],
    "logo": quants[:,3],
    "country": quants[:,4],
    "city": quants[:, 5]
}

with open("portugal_teams.pkl", "wb") as handle:    
    pickle.dump(dict, handle,
                protocol=pickle.HIGHEST_PROTOCOL) 
